# **Sternhaufen mit Gaia und Python**

## Willkommen!

In diesem Notebook werden wir Farben-Helligkeits-Diagramme (FHDs) von Sternhaufen erstellen und analysieren, um die Entfernung und das Alter dieser Sternhaufen abzuschätzen. Dafür nutzen wir Daten des Weltraumteleskops Gaia der ESA.

Die Nutzung des Notebooks werden wir Dir live vorführen. Es ist hier nicht jedes Detail schriftlich beschrieben. Melde Dich einfach, wenn etwas nicht klappt!

  - Link zu Jürgens Folien ?
  - python env auswählen?
  - Restart und clear outputs hinweisen (wie das in Jupyterlab aussieht...)?

## Teil 1: kleines Python-Tutorial

Damit du den Code in den nächsten Teilen nachvollziehen kannst, fangen wir mit einer kleinen Einleitung zu Python an.


Nutze **control-enter** um eine Zelle auszuführen!

In [ ]:
print("Hallo Welt!")

In [ ]:
# Alles hinter einem # ist ein Kommentar, der von Python ignoriert wird.

# Das hier ist ein "String", also eine Zeichenkette, die von Python als Text interpretiert wird:
t = "Hallo Welt!"

# In einem Notebook kannst du das print() oft weglassen, der Wert des letzten "Ausdrucks" einer Zelle wird automatisch ausgegeben, so wie hier:
t

In [ ]:
a = 6
b = 3
print("Kleine Rechnung:", a + b - 5.2)

# Wenn gewünscht kann man genau bestimmen, wie Zahlen formatiert werden sollen.
# Die modernste und eleganteste Lösung dafür in Python ist der sogenannte f-String:
print(f"Noch ne Rechnung: { a+b :.5f}") # Das .5f bedeutet, dass die Zahl mit 3 Nachkommastellen ausgegeben werden soll.

In [ ]:
# Variablen bleiben nach der Ausführung von Zellen erhalten!

a / b

Für praktisch all Aufgaben gibt es in Python "Module", das sind Programmierbibliotheken. Um sie zu nutzen, muss man sie vorher einmal *importieren*.

Erstes Beispiel: für numerische Rechnungen nutzt man **numpy**, und importiert es als np (... ist kürzer zu schreiben), so: 

In [ ]:
import numpy as np

# Alles, was zu numpy gehört, fängt nun mit "np." an:
x = 2.0
y = np.sqrt(x) # sqrt steht für "square root"

print("Die Quadratwurzel von", x, "ist ungefähr", y)

print("Pi ist ungefähr", np.pi)


Die wichtigste Eigenschaft von numpy ist das rechnen mit sogenannten "Arrays". Ein Array ist eine Art Liste (oder Tabelle) von Zahlen.

So können wir ein Array selber bauen:

In [ ]:
a = np.array([1.5, 4.2, 2.0, 1.0])

Und jetzt kommt der Clou: wenn man mit einem Array rechnet, werden viele Operationen automatisch auf *alle* Zahlen dieses Arrays angewandt. Das Ergebnis so einer Rechnung ist oft selber ein Array. Probiere folgende Beispiele aus:

In [ ]:
b = a + 2.0  # b ist nun auch ein Array mit 4 Zahlen...
print(b)

b = np.sqrt(a)  # ... und hier ist b die Quadratwurzel von a
print(b)

Andere Funktionen von numpy können ein Array "zusammenfassen", und nur eine Zahl ausgeben. Zum Beispiel:

In [ ]:
print(np.mean(a)) # Mittelwert von a

c = np.max(b) # Maximalwert von b
print(c)

Als nächstes Beispiel schauen wir uns kurz das Modul **matplotlib** an, mit dem man Diagramme erstellen kann. Mit dieser Bibliothek können alle Aspekte eines Diagramms bis ins letzte Detail kontrolliert werden, und trotzdem bleibt das erstellen von einfachen Diagrammen so unkompliziert wie möglich.



In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt # Wir werden matplotlib immer so importieren

# Matplotlib funktioniert mit numpy Arrays, wir bauen uns also ein Beispiel:
x = np.linspace(0, 5, 50) # so bekommt man ein Array mit 50 regelmässig verteilte Zahlen zwischen 0 und 5
y = np.sqrt(x) # Das Array y enthält also die Quadratwurzeln der Werte von x

# Und jetzt matplotlib:
plt.figure() # Lege eine neues Diagramm an
plt.plot(x, y, "b.") # Zeichne (x, y) als blaue Punkte ein ("b." steht für "blue dot")
plt.show() # Zeige das Diagramm an

Matplotlib-Diagramme sind interaktiv. Bewege die Maus über das Diagramm, und Klicke auf das Kreuz-Symbol oben links. Jetzt kannst du mit der linken Maustaste die Achsen verschieben, und mit der rechten Maustaste die Skala der Achsen ändern.

Spiele ein wenig damit herum:

 - Ersetze mal `sqrt` durch `sin`.
 - Ersetzte `"b."` durch `"r-"`, oder lasse das ganz weg.
 - Du kannst die Funktion `plt.plot` auch mehrmals aufrufen (natürlich am besten mit unterschiedlichem Inhalt).
 - Füge die Zeile `plt.xlabel("Das ist die x-Achse")` ein, vor dem `plt.show()`.

## Teil 2: Laden der Daten

In diesem Teil laden wir die Gaia-Daten eines Sternhaufens aus dem Internet. 

Zunächst importieren wir jetzt alle Module, die wir heute benötigen werden. Führe die folgende Zelle einfach einmal aus.

In [ ]:
import warnings

import urllib.request
import pathlib
import pickle

import numpy as np
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

import pyvo

import astropy
from astropy import units as u

from ipyaladin import Aladin, Marker

In [ ]:
sternhaufen_name = "M11"

In [ ]:
from ipyaladin import __version__, __aladin_lite_version__
print("version:", __version__, "running Aladin Lite:", __aladin_lite_version__)

In [ ]:
aladin = Aladin(target=sternhaufen_name, fov=3.0, survey="P/DSS2/color", height=600)
aladin

In [ ]:
markers = []
for i in range(1, 11):
    name = f"M{i}"
    markers.append(
        Marker(
            position=name,
            title=name,
            # the title and description can be written as plain text or as html elements
            description=(
                '<a href="https://simbad.cds.unistra.fr/simbad/'
                f'sim-basic?Ident={name}&submit=SIMBAD+search"> '
                "Read more on SIMBAD</a>"
            ),
        )
    )
aladin.add_markers(markers, name="M1-M10", color="pink", shape="cross", source_size=15)
aladin.target = "M1"
aladin.fov = 0.2

In [ ]:

# Die Himmelskoordinaten
# Mit Hilfe von astropy können wir diese direkt herausfinden!
sternhaufen_position = astropy.coordinates.SkyCoord.from_name(sternhaufen_name)
dec_grad = sternhaufen_position.dec.to_value(u.deg)
ra_grad = sternhaufen_position.ra.to_value(u.deg)

# Der Radius, innerhalb dessens wir den Katalog laden wollen:
radius_grad = 0.5 # Dieser Radius ist in Grad angegeben.
radius_grad = 0.1 # Dieser Radius ist in Grad angegeben.


print("Die Koordinaten (in Grad): DEC = ", dec_grad, ", und RA = ", ra_grad)
# Du könntest das mit dem vergleichten, was Wikipedia für M4 angibt.

# Jetzt geben wir an, woher wir die Daten laden wollen:
tap_service = pyvo.dal.TAPService("https://gaia.ari.uni-heidelberg.de/tap")

# Und wir beschreiben, in einer speziellen Sprache, welche Daten wir wollen.
# In den ersten Zeilen, nach SELECT, geben wir die namen der Spalten an,
# die wir brauchen.
# Dann folgt die Angabe der Himmelsregion um den Sternhaufen.
tap_anfrage = """
SELECT ra, dec, pmra, pmdec,
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, parallax, parallax_error
FROM gaiadr2.gaia_source
WHERE 1 = CONTAINS(POINT('ICRS', ra, dec),
                   CIRCLE('ICRS', {}, {}, {}))
AND phot_g_mean_mag < 17.0
""".format(ra_grad, dec_grad, radius_grad)
print("Die Anfrage an der Server lautet:")
print(tap_anfrage)

# Nun stellen wir diese Anfrage, und bekommen (wenn alles klappt) die Daten.
print("Die Anfrage wird nun gestellt.")
print("Es kann einen Moment dauern, bis die Daten angekommen sind.")
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", module='astropy.*')
    tap_daten = tap_service.run_sync(tap_anfrage)

katalog = tap_daten.to_table()

print("Wir haben den Katalog bekommen. Zusammenfassung:")
print(katalog.info)
# Diese Zusammenfassung ist nützlich, daher schreiben wir sie in eine Datei:
with open("info_katalog.txt", "w") as datei:
    datei.write(str(katalog.info))

# Jetzt schreiben wir den Katalog in eine einfache Text-Datei,
# als eine große Tabelle, im Format "tab-separated values", oft tsv genannt.
katalog.write(f"katalog_um_{sternhaufen_name}.tsv", format="ascii.tab", overwrite=True)
print("Fertig!")

In [ ]:
aladin.add_table(
    katalog,
    shape="circle",
    color="red",
    ra_field="ra",
    dec_field="dec",
)

In [ ]:


# Wir laden den Katalog so wie in der Einleitung gezeigt:
katalog = np.genfromtxt(f"katalog_um_{sternhaufen_name}.tsv", delimiter="\t", names=True)

# Und erstellen ein Diagram mit Matplotlib, folgendes wird gleich besprochen.
plt.figure(figsize=(6, 6))
plt.plot(katalog["pmra"], katalog["pmdec"],
    marker=".", markersize=1, linestyle="None")

plt.title(f"Gaia-Eigenbewegungen im Himmelsauschnitt um {sternhaufen_name}")
plt.xlabel("Eigenbewegung in RA [welche Einheit?]")
plt.ylabel("Eigenbewegung in Dec [welche Einheit?]")

plt.tight_layout()
plt.show()

In [ ]:

#katalog = np.genfromtxt(f"katalog_{sternhaufen_name}.tsv", delimiter="\t", names=True)
katalog = np.genfromtxt(f"katalog_um_{sternhaufen_name}.tsv", delimiter="\t", names=True)
#katalog = np.delete(katalog, 0) # Entfernen der nan-Zeile, die durch die
# Striche unter den Spaltennamen ensteht.

# Als erstes geben wir mal die Namen der Spalten aus:
print("Namen der Spalten:", katalog.dtype.names)
print("Anzahl der Sterne:", len(katalog))

plt.figure()
plt.plot(
    katalog["phot_bp_mean_mag"] - katalog["phot_rp_mean_mag"], # Farbe: B - R
    katalog["phot_rp_mean_mag"], # Scheinbare Helligkeit: R
    label="M4",
    marker=".", markersize=2, linestyle="None", alpha=0.3, rasterized=True
    )

# Wie schon im Modul 2 gemacht ändern wir die Richtung der y-Achse:
plt.gca().invert_yaxis()

# Und Beschriften das Diagram:
plt.title(f"Gaia-FHD von {sternhaufen_name}")
plt.xlabel("Farbe B - R [mag]")
plt.ylabel("Scheinbare Helligkeit R [mag]")
plt.legend()

# Optional: setzte folgende Zeile direkt vor show() oder savefig(),
# dann wird der Abstand zwischen Beschrifung und Diagram optimiert:
plt.tight_layout()

# Und zum Schluss eines von
plt.show()


In [ ]:
# Download the file containing all the isochrones (about 48 MB):

local_isochrones_filepath = pathlib.Path("MIST_Gaia_isochrones.pickle")

isochrones_url = "https://uni-bonn.sciebo.de/s/twnf8ZiN66RySCT/download/MIST_Gaia_isochrones.pickle"
if not local_isochrones_filepath.exists():
    urllib.request.urlretrieve(isochrones_url, local_isochrones_filepath)

# Read this file with the following code (see below for explanations):
with open(local_isochrones_filepath, 'rb') as f:
    (isochrones, vrots, metallicities, logages) = pickle.load(f)

In [ ]:
def extinction_law(A_V):
    """
    Returns extinction values for the different Gaia filters, based on some reference total extinction A_V

    The particular values we use here are from Wang and Chen (2019) ApJ 877 116:
    https://iopscience.iop.org/article/10.3847/1538-4357/ab1c61
    """
    #return {"g": ref_ext*3.30, "r": ref_ext*2.31, "i":ref_ext*1.71 } # Table 2 of Yuan et al. (2013) https://doi.org/10.1093/mnras/stt039
    return {"G": A_V*0.8, "RP": A_V*0.589, "BP":A_V*1.002 } # Table 3 of Wang and Chen (2019)



def mag_absolute_to_apparent(m, distance, extinction):
    """
    Function that corrects absolute magnitudes by distance modulus and extinction, to get apparent magnitudes
    
    m: absolute magnitude(s)
    distance: in pc
    extinction: in mag
    """
    return m + 5.0*np.log10(distance) - 5 + extinction 


def get_apparent_isochrone(vrot_index, metallicity_index, age_index, distance=10, A_V=0):
    """
    Helper function to get a specific isochrone at a specific distance and total extinction
    """
    
    # Computing the extinctions:
    extinctions = extinction_law(A_V)
    
    # Obtaining the corrected magnitudes:
    iso_app_g = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["G"], distance, extinctions["G"])
    iso_app_r = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["BP"], distance, extinctions["BP"])
    iso_app_b = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["RP"], distance, extinctions["RP"])
    
    # A fancy string describing the isochrone, with some LaTeX formatting:
    label_string = f"Age {(10.0**logages[age_index])/(1e9):.2f} Gyr, " \
        + f"[Fe/H] = {metallicities[metallicity_index]:.2f}, " \
        + r"$v/v_{\mathrm{crit}}$ = " + f"{vrots[vrot_index]}, " \
        + f"$d$ = {distance:.0f} pc, " \
        + r"$A_{\mathrm{V}}$ = " + f"{A_V:.2f}"

    return (iso_app_g, iso_app_r, iso_app_b, label_string)



In [ ]:
# Initial position of sliders:
init_distance = 10.0
init_age_index = 50
init_metallicity_index = 12
init_A_V = 0.0
init_vrot_index = 0

fig, ax = plt.subplots(figsize=(10, 6))
fig.subplots_adjust(left=0.30, bottom=0.1) # making room for the sliders

# Plotting the observed data
cbar = ax.plot(
    katalog["phot_bp_mean_mag"] - katalog["phot_rp_mean_mag"], # Farbe: B - R
    katalog["phot_rp_mean_mag"], # Scheinbare Helligkeit: R
    marker=".", markersize=2, linestyle="None", alpha=0.3, rasterized=True
)

# Plotting the isochrone
(g, bp, rp, label_string) = get_apparent_isochrone(init_vrot_index, init_metallicity_index, init_age_index, init_distance, init_A_V)
line, = ax.plot(bp-rp, rp, color="red", linewidth=1.0) 
text = ax.text(0.5, 0.95, label_string, color="red", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

# The function to be called anytime a slider or botton is changed
def update(val):
    vrot_index = vrot_button.index_selected
    metallicity_index = metallicity_slider.val
    age_index = age_slider.val
    distance = dist_slider.val
    A_V = ext_slider.val
    (g, bp, rp, label_string) = get_apparent_isochrone(vrot_index, metallicity_index, age_index, distance, A_V)
    line.set_xdata(bp-rp)
    line.set_ydata(rp)
    text.set_text(label_string)
    fig.canvas.draw_idle()

# Slider to control the distance
axdist = fig.add_axes([0.05, 0.25, 0.0225, 0.6])
dist_slider = matplotlib.widgets.Slider(
    ax=axdist,
    label=r"$d$ [pc]",
    valmin=10,
    valmax=10000,
    valinit=init_distance,
    orientation="vertical"
)

# Slider to control the age
axage = fig.add_axes([0.09, 0.25, 0.0225, 0.6])
age_slider = matplotlib.widgets.Slider(
    ax=axage,
    label=r"$i_{\mathrm{Age}}$",
    valmin=0,
    valmax=len(logages)-1,
    valstep=1,
    valinit=init_age_index,
    orientation="vertical"
)

# Slider to control the metallicity
axmetal = fig.add_axes([0.13, 0.25, 0.0225, 0.6])
metallicity_slider = matplotlib.widgets.Slider(
    ax=axmetal,
    label=r"$i_{\mathrm{[Fe/H]}}$",
    valmin=0,
    valmax=len(metallicities)-1,
    valstep=1,
    valinit=init_metallicity_index,
    orientation="vertical"
)

# Slider to control the extinction
axext = fig.add_axes([0.17, 0.25, 0.0225, 0.6])
ext_slider = matplotlib.widgets.Slider(
    ax=axext,
    label=r"$A_{\mathrm{V}}$",
    valmin=0.0,
    valmax=3.0,
    valinit=init_A_V,
    orientation="vertical"
)

# Radio buttons to control the rotation
axvrot = fig.add_axes([0.05, 0.1, 0.14, 0.08])
vrot_button = matplotlib.widgets.RadioButtons(
    ax=axvrot, 
    labels=("No rotation", r"$v/v_{\mathrm{crit}} = 0.4$"),
    active=init_vrot_index
)

# Register the update function with each slider and button:
age_slider.on_changed(update)
dist_slider.on_changed(update)
metallicity_slider.on_changed(update)
ext_slider.on_changed(update)
vrot_button.on_clicked(update)

# Finalize plot
ax.invert_yaxis()
ax.set_xlabel("$B$ - $R$")
ax.set_ylabel("$R$")
ax.set_title(f"Gaia-FHD von {sternhaufen_name}")
plt.show()

## Teil 5: Zusammenführen der Ergebnisse!


Spalte:
Link zu Google doc
https://docs.google.com/spreadsheets/d/1W0GKbjyw9aVHF4K5B1VOvJqouxk3H_Mv9EgN-Bxl35Y/



- Nummer
- Kugelsternahfuen oder offener Haufen (hmm, oder das gar nicht erst erklären)
- Name Sternhaufen
- Extinction
- Galactische breite
- Entfernung in kpc durch Parallaxen, wenn möglich?
- Entfernung in kpc (von Dir bestimmt)
- Alter in Gyr (von Dir bestimmt)


Analyse:
Kugelsternhaufen vs Sternhaufen (alter! und wie sie aussehen, entfernung)


## Notlösungen, falls etwas nicht funktioniert

sciebo link zu Gaia daten-paket

link zu einem read-only google doc